# MNIST Classification with a Softmax Head
This notebook mirrors the LDA-based workflow, but uses a standard softmax classifier head on top of the encoder.


### Setup


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device =', device)


device = cuda


### Data


In [3]:
tfm = transforms.ToTensor()
train_ds = datasets.MNIST(root='./data', train=True, transform=tfm, download=True)
test_ds  = datasets.MNIST(root='./data', train=False, transform=tfm, download=True)
train_ld = DataLoader(train_ds, batch_size=256, shuffle=True, num_workers=2, pin_memory=True)
test_ld  = DataLoader(test_ds,  batch_size=1024, shuffle=False, num_workers=2, pin_memory=True)
len(train_ds), len(test_ds)


(60000, 10000)

### Model: encoder + softmax head


In [4]:
class Encoder(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 256), nn.ReLU(inplace=True),
            nn.Linear(256, 64), nn.ReLU(inplace=True),
            nn.Linear(64, dim),
        )

    def forward(self, x):
        return self.net(x)

class SoftmaxHead(nn.Module):
    def __init__(self, D, C):
        super().__init__()
        self.linear = nn.Linear(D, C)

    def forward(self, z):
        return self.linear(z)

class DeepClassifier(nn.Module):
    def __init__(self, C, D):
        super().__init__()
        self.encoder = Encoder(D)
        self.head = SoftmaxHead(D, C)

    def forward(self, x):
        z = self.encoder(x)
        return self.head(z)


### Train & Eval


In [5]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    ok = tot = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        ok += (logits.argmax(1) == y).sum().item()
        tot += y.size(0)
    return ok / tot

model = DeepClassifier(C=10, D=9).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(1, 16):
    model.train()
    loss_sum = acc_sum = n_sum = 0
    for x, y in train_ld:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = loss_fn(logits, y)

        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()

        with torch.no_grad():
            pred = logits.argmax(1)
            acc_sum += (pred == y).sum().item()
            n_sum += y.size(0)
            loss_sum += loss.item() * y.size(0)

    tr_acc = acc_sum / n_sum
    te_acc = evaluate(model, test_ld)
    print(f"[{epoch:02d}] train loss={loss_sum/n_sum:.4f} acc={tr_acc:.4f} | test acc={te_acc:.4f}")


[01] train loss=0.6153 acc=0.8251 | test acc=0.9282
[02] train loss=0.2075 acc=0.9422 | test acc=0.9517
[03] train loss=0.1436 acc=0.9584 | test acc=0.9563
[04] train loss=0.1133 acc=0.9667 | test acc=0.9655
[05] train loss=0.0867 acc=0.9742 | test acc=0.9696
[06] train loss=0.0721 acc=0.9784 | test acc=0.9710
[07] train loss=0.0585 acc=0.9825 | test acc=0.9732
[08] train loss=0.0500 acc=0.9847 | test acc=0.9722
[09] train loss=0.0414 acc=0.9877 | test acc=0.9761
[10] train loss=0.0325 acc=0.9904 | test acc=0.9761
[11] train loss=0.0272 acc=0.9915 | test acc=0.9758
[12] train loss=0.0214 acc=0.9939 | test acc=0.9758
[13] train loss=0.0174 acc=0.9949 | test acc=0.9759
[14] train loss=0.0160 acc=0.9952 | test acc=0.9744
[15] train loss=0.0110 acc=0.9972 | test acc=0.9791


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

model.eval()


### Collect a Small Random Subset of Embeddings


In [ ]:
emb_list, y_list = [], []
max_points = int(len(train_ld.dataset) * 0.05)

with torch.no_grad():
    for X, y in train_ld:
        X, y = X.to(device), y.to(device)
        z = model.encoder(X)
        emb_list.append(z)
        y_list.append(y)
        if sum(t.shape[0] for t in emb_list) >= max_points:
            break

Z = torch.cat(emb_list, dim=0).cpu().numpy()[:max_points]
Y = torch.cat(y_list, dim=0).cpu().numpy()[:max_points]
Z.shape, Y.shape


### Inspect Learned Classifier Weights


In [ ]:
W = model.head.linear.weight.detach().cpu()
b = model.head.linear.bias.detach().cpu()
W.shape, b.shape


### Visualise Embeddings


In [ ]:
plt.figure(figsize=(7, 6))
cmap = plt.get_cmap("tab10", 10)
markers = ['o','s','^','v','<','>','P','X','D','*']

for c in range(10):
    idx = (Y == c)
    plt.scatter(Z[idx, 0], Z[idx, 1], s=8, alpha=0.6,
                c=[cmap(c)], marker=markers[c], label=f"{c}")

plt.title("Deep softmax head on MNIST: sampled training embeddings")
plt.xlabel("Embedding dim 1")
plt.ylabel("Embedding dim 2")
plt.legend(fontsize=8, ncol=2, frameon=False)
plt.tight_layout()
plt.show()
